In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression

In [2]:
df = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/frauddetectionsmall.csv')


In [3]:
display(df)

In [4]:
df.printSchema()

In [5]:
#select data column
df2 = df.select("type", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", (col("isFraud").cast("Int").alias("label")))
display(df2)

In [6]:
#Split the data
splits = df2.randomSplit([0.7, 0.3])
train = splits[0]

In [7]:
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()

In [8]:
print "Training Rows:", train_rows, " Testing Rows:", test_rows
train.show(5)
test.show(5)